In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install protobuf==3.20.3
!pip install --upgrade tensorflow==2.12.0

In [ ]:
import os
import json
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
BASE = "/kaggle/input/the-blind-flight-synapse-drive-ps-1/SynapseDrive_Dataset"

train_img_path = BASE + "/train/images"
train_label_path = BASE + "/train/labels"
test_img_path  = BASE + "/test/images"
test_vel_path  = BASE + "/test/velocities"


In [ ]:
tiles = []
labels = []

for img_file in sorted(os.listdir(train_img_path)):
    
    img_id = img_file.replace(".png","")
    
    img = cv2.imread(f"{train_img_path}/{img_file}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    h, w, _ = img.shape
    tile_h = h // 20
    tile_w = w // 20
    
    label_json = json.load(open(f"{train_label_path}/{img_id}.json"))
    grid = np.array(label_json["grid"])
    
    for i in range(20):
        for j in range(20):
            tile = img[i*tile_h:(i+1)*tile_h, j*tile_w:(j+1)*tile_w]
            tile = cv2.resize(tile,(32,32))
            tiles.append(tile)
            labels.append(grid[i][j])

tiles = np.array(tiles)
labels = np.array(labels)

print("Dataset:", tiles.shape, labels.shape)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    tiles, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

X_train = X_train / 255.0
X_val   = X_val / 255.0


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:
model = models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(128,(3,3),activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),

    layers.Dense(128,activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(5,activation='softmax')
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val,y_val),
    epochs=20,
    batch_size=128,
    verbose=1
)

In [ ]:
model.save("/kaggle/working/tile_cnn.h5")

In [ ]:
def predict_grid(img_path):
    try:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        h,w,_ = img.shape
        tile_h = h//20
        tile_w = w//20

        tiles = []

        for i in range(20):
            for j in range(20):
                tile = img[i*tile_h:(i+1)*tile_h, j*tile_w:(j+1)*tile_w]
                tile = cv2.resize(tile,(32,32)) / 255.0
                tiles.append(tile)

        tiles = np.array(tiles)

        preds = model.predict(tiles, verbose=0)

        grid_pred = preds.argmax(axis=1).reshape(20,20)
        prob_map  = preds.reshape(20,20,5)


        if not np.any(grid_pred==3):
            s = np.unravel_index(np.argmax(prob_map[:,:,3]), (20,20))
            grid_pred[s] = 3


        if not np.any(grid_pred==4):
            g = np.unravel_index(np.argmax(prob_map[:,:,4]), (20,20))
            grid_pred[g] = 4


        if np.sum(grid_pred==3) > 1:
            best = np.unravel_index(np.argmax(prob_map[:,:,3]), (20,20))
            grid_pred[grid_pred==3] = 0
            grid_pred[best] = 3


        if np.sum(grid_pred==4) > 1:
            best = np.unravel_index(np.argmax(prob_map[:,:,4]), (20,20))
            grid_pred[grid_pred==4] = 0
            grid_pred[best] = 4

        return grid_pred.astype(int)

    except Exception as e:
        print("Predict_grid failed for:", img_path)
        print("Error:", e)

        
        fallback = np.zeros((20,20),dtype=int)
        fallback[1,1] = 3
        fallback[18,18] = 4
        return fallback

In [ ]:
def detect_terrain(image):
    mean_val = image.mean()
    if mean_val > 180:
        return "desert"
    elif mean_val < 100:
        return "lab"
    return "forest"

In [ ]:
BASE_COST = {
    "lab":    {0:1.0, 2:3.0, 3:1.0, 4:2.0},
    "forest": {0:1.5, 2:2.8, 3:1.5, 4:2.5},
    "desert": {0:1.2, 2:3.7, 3:1.2, 4:2.2},
}

In [ ]:
from heapq import heappush, heappop

DIRS = [(0,1,'r'),(0,-1,'l'),(1,0,'d'),(-1,0,'u')]

def a_star(grid, boosts, terrain):

    
    start_pts = np.argwhere(grid==3)
    if len(start_pts)==0:
        
        start = (1,1)
    else:
        start = tuple(start_pts[0])

   
    goal_pts = np.argwhere(grid==4)
    if len(goal_pts)==0:
        
        goal = (18,18)
    else:
        goal = tuple(goal_pts[0])

    ...

    
    pq = []
    heappush(pq,(0,start,""))
    visited=set()
    
    while pq:
        cost,(x,y),path = heappop(pq)
        
        if (x,y) in visited:
            continue
        visited.add((x,y))
        
        if (x,y)==goal:
            return path
        
        for dx,dy,ch in DIRS:
            nx,ny = x+dx,y+dy
            if 0<=nx<20 and 0<=ny<20:
                c = grid[nx][ny]
                if c==1:
                    continue
                
                step = BASE_COST[terrain].get(c,5.0) - boosts[nx][ny]
                heappush(pq,(cost+step,(nx,ny),path+ch))
    
    return ""

In [ ]:
results = []

for file in sorted(os.listdir(test_img_path)):
    
    img_id = file.replace(".png","")
    
    img = cv2.imread(f"{test_img_path}/{file}")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    terrain = detect_terrain(gray)
    grid = predict_grid(f"{test_img_path}/{file}")
    
    boosts = np.array(
        json.load(open(f"{test_vel_path}/{img_id}.json"))["boost"]
    )
    
    path = a_star(grid, boosts, terrain)
    
    results.append([img_id,path])

df = pd.DataFrame(results, columns=["image_id","path"])
df.to_csv("/kaggle/working/submission.csv", index=False)

df.head()

In [ ]:
df[df["path"].isna()]

In [ ]:
df["path"] = df["path"].fillna("")
df.loc[df["path"]=="", "path"] = "r"  

In [ ]:
print("Unique IDs:", df["image_id"].nunique())
print("Rows:", len(df))

In [ ]:
len(os.listdir(test_img_path))

In [ ]:
for file in sorted(os.listdir(test_img_path)):

    img_id = file.replace(".png","")

    try:
        img = cv2.imread(f"{test_img_path}/{file}")
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        terrain = detect_terrain(gray)
        grid = predict_grid(f"{test_img_path}/{file}")

        if grid is None:
            print("Grid None → fallback", img_id)
            grid = np.zeros((20,20), dtype=int)
            grid[1,1]=3
            grid[18,18]=4

        boosts = np.array(json.load(open(f"{test_vel_path}/{img_id}.json"))["boost"])

        path = a_star(grid, boosts, terrain)

        if path is None or path=="":
            print("Empty path → fallback", img_id)
            path = "r" * 5  

    except Exception as e:
        print("Loop failed for", img_id, "→ fallback")
        print(e)
        path = "r" * 5

    results.append([img_id, path])

In [ ]:
df = pd.DataFrame(results, columns=["image_id","path"])

df["path"] = df["path"].astype(str)
df["path"] = df["path"].fillna("r")

df.to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
results_dict = {rid: rpath for rid, rpath in results}  

safe_rows = []

for file in sorted(os.listdir(test_img_path)):
    img_id = file.replace(".png","")

    path = results_dict.get(img_id, "")

    if path is None:
        path = ""

    path = str(path)

    if path.strip() == "" or path.lower() in ["nan", "none"]:
        
        path = "r"     

    safe_rows.append([img_id, path])

df = pd.DataFrame(safe_rows, columns=["image_id","path"])


df["image_id"] = df["image_id"].astype(str)
df["path"] = df["path"].astype(str)
df["path"] = df["path"].fillna("r")
df.loc[df["path"].str.strip()=="", "path"] = "r"

print("NULL COUNT:")
print(df.isna().sum())
print(df.head())

In [ ]:
df.to_csv("/kaggle/working/submission.csv", index=False)